In [1]:
import pandas as pd
import geopandas as gpd
from shapely import Point
from math import radians, cos, sin, asin, sqrt
import dask.dataframe as dd
from pathlib import Path
import sys, os
sys.path.append(os.path.join(os.path.abspath(""), '..', 'pipeline'))

In [2]:
import distances
import counterglow_matches as cg

In [3]:
# Enter the SIC Code of interest here
SIC_CODE = '0291'

# radius that is used as a limit of distance between two sets of coordinates
RADIUS_KM = 0.3048

In [4]:
# establishing the file path in such a way to work universally
here = Path().resolve()

In [5]:
# load counterglow dataset
# df_counterglow = pd.read_csv("../data/Counterglow+Facility+List+Complete.csv")

df_counterglow = pd.read_csv(here.parent / "data/Counterglow+Facility+List+Complete.csv")

In [6]:
# filter counterglow dataset to have poultry/chicken only

# Chickens & Other Birds (Meat)
# Chickens (Meat)

df_counterglow_poultry = df_counterglow[(df_counterglow['Farm Type'] == 'Chickens & Other Birds (Meat)') |
                                        (df_counterglow['Farm Type'] == 'Chickens (Meat)')]

In [7]:
# list of states we want to observe
states = ['IA', 'OK', 'MO', 'OK', 'AL', 'LA', 'MS', 'IL', 'IN', 'OH', 'KY', 'TN', 'AR', 'NC', 'SC', 'GA']

# all the dataframes 
all_df_list = []

## SIC Code 0291 Exploration: General Farms, Primarily Livestock and Animal Specialties.

In [10]:
# To see how many matches exist within the dataframe consisting of SIC Codes containing SIC_CODE of interest

path = "data/code_" + SIC_CODE + ".csv"
df_sic = pd.read_csv(here.parent / path)
  
infogroup_sic_dict, counterglow_dict = cg.infogroup_counterglow_dict(df_sic, df_counterglow_poultry, states, RADIUS_KM)


NC: Total number of matches: 1


In [71]:
# loop through the infogroup dictionary, each key value matches to a dictionary
# grab the dictionary and append it to all_df_list

all_df_list = list_of_dataframes(infogroup_sic_dict, all_df_list)

In [72]:
len(all_df_list)

1

In [12]:
# View the keys of the dictionary. Int his case, they are the states
infogroup_sic_dict.keys()

dict_keys(['NC'])

In [ ]:
# To analyze a specific state, input one of the two letter states in the keys in the above
# cell into the line below. Uncomment and run

# infogroup_sic_dict['TWO LETTER STATE HERE']

## To CSV

In [98]:
### concat all DataFrames & convert to csv

infogroup_concat = pd.concat(all_df_list)

FILEPATH = "../data/infogroup_matches.csv"
infogroup_concat.to_csv(FILEPATH) # save as CSV file